# SQL

El coronavirus tomó al mundo entero por sorpresa, cambiando la rutina diaria de todos y todas. Los habitantes de las ciudades ya no pasaban su tiempo libre fuera, yendo a cafés y centros comerciales; sino que más gente se quedaba en casa, leyendo libros. Eso atrajo la atención de las startups (empresas emergentes) que se apresuraron a desarrollar nuevas aplicaciones para los amantes de los libros.

Te han dado una base de datos de uno de los servicios que compiten en este mercado. Contiene datos sobre libros, editoriales, autores y calificaciones de clientes y reseñas de libros. Esta información se utilizará para generar una propuesta de valor para un nuevo producto.

## **Objetivo**

-   Analizar tendencias y patrones en el mercado de lectura.
  
## **Etapas del Proyecto**

### 1. **Preprocesamiento de Datos**

Objetivo: Imprimir y estudiar los datos mediante consultas sq.

- Imprimir de cada tabla, las 10 primeras filas.

### 2. **Análisis Exploratorio de Datos (EDA)**

Objetivo:  Entender y analizar el comportamiento de los usuarios, identificar editoriales más activas y autores con calificaciones más  altas.
    
- Encuentra el número de libros publicados después del 1 de enero de 2000.
- Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.
- Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).
- Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.
- Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros
    

### 3. **Conclusiones**

- Conclusiones con respecto a la etapa EDA.

In [61]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML

In [12]:
# Conexion con la BASE DE DATOS

from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],db_config['pwd'], db_config['host'], db_config['port'],db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})


# 1. **Preprocesamiento de Datos** : 
### Cargar, y estudiar los datos para el análisis y creación de consultas sql.

In [13]:
libros = pd.read_sql('SELECT * FROM books', con = engine)
autores = pd.read_sql('SELECT * FROM authors', con = engine)
editoriales = pd.read_sql('SELECT * FROM publishers', con = engine)
calificacion = pd.read_sql('SELECT * FROM ratings', con = engine)
reviews = pd.read_sql('SELECT * FROM reviews', con = engine)

In [14]:
libros.head(n=10)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


In [18]:
autores.head(n=10)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
5,6,Alan Paton
6,7,Albert Camus/Justin O'Brien
7,8,Aldous Huxley
8,9,Aldous Huxley/Christopher Hitchens
9,10,Aleksandr Solzhenitsyn/H.T. Willetts


In [22]:
editoriales.head(n=10)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
5,6,Aladdin
6,7,Aladdin Paperbacks
7,8,Albin Michel
8,9,Alfred A. Knopf
9,10,Alfred A. Knopf Books for Young Readers


In [26]:
calificacion.head(n=10)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4


In [31]:
reviews.head(n=10)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With pe...
7,8,4,abbottjames,Within enough mother. There at system full rec...
8,9,5,npowers,Thank now focus realize economy focus fly. Ite...
9,10,5,staylor,Game push lot reduce where remember. Including...


In [106]:
# Evaluando usuarios de mi dataframe

query = """
    
    SELECT COUNT(DISTINCT username) AS total_usuarios
    FROM reviews;
"""
usuarios_total = pd.read_sql(query, con=engine)
display(usuarios_total)

# Valor mas alto de calificacion

query1 = """
    SELECT MAX(rating) AS valor_mas_alto
    FROM ratings;
"""
calificacion_max = pd.read_sql(query1, con=engine)
display(calificacion_max)

# Explorando otros datos

query2 = """
    
    SELECT COUNT(DISTINCT title) AS titulos_de_libros
    FROM books;
"""
titulos_libros = pd.read_sql(query2, con=engine)
display(titulos_libros)

,total_usuarios
0,160


,valor_mas_alto
0,5


,titulos_de_libros
0,999


# 2. **Análisis Exploratorio de Datos (EDA)**
### Entender y analizar el comportamiento de los usuarios, identificar editoriales más activas y autores con calificaciones más  altas.

In [110]:
def execute_query(query, engine, title=None):
    """
    Ejecuta una consulta SQL y devuelve los resultados en un DataFrame, mostrando un título antes de los resultados.
    
    Parámetros:
        query (str): La consulta SQL a ejecutar.
        engine: Conexión SQLAlchemy a la base de datos.
        title (str, opcional): Título a mostrar antes de los resultados.
    
    Retorno:
        pd.DataFrame: DataFrame con los resultados de la consulta.
    """
    try:
        # Mostrar título en negrita y tamaño más grande
        display(HTML(f"<h2 style='color: #2C3E50; font-size: 24px; font-weight: bold;'>{title}</h2>"))
        
        df = pd.read_sql(query, con=engine)
        display(df)
        return df
    except Exception as e:
        print(f"Error al ejecutar la consulta: {e}")
        return pd.DataFrame()


query = """
    SELECT 
        COUNT(books.book_id) AS total_de_libros
    FROM 
        books
    WHERE 
        publication_date >'2000-01-01';
"""
num_libros = execute_query(query, engine, title="1.Encuentra el número de libros publicados después del 1 de enero de 2000.")


query1 = """
    WITH reviews_count AS (
        SELECT 
            username,
            COUNT(review_id) OVER (PARTITION BY username) AS reviews_cantidad
        FROM 
            reviews
    )
    SELECT DISTINCT * FROM reviews_count;
    """
reviews_avg = execute_query(query1, engine, title="2.Número de reseñas por usuarios")

query4 = """
    WITH reviews_count AS (
        SELECT 
            COUNT(review_id) AS reviews_cantidad
        FROM 
            reviews
    )
    SELECT DISTINCT * FROM reviews_count;
    """
reviews_total = execute_query(query4, engine, title="3.Encuentra el número de reseñas de usuarios.")


query2 = """
    WITH book_avg_rating AS (
        SELECT 
            books.title AS nombre_libro,
            AVG(ratings.rating) OVER (PARTITION BY books.title) AS promedio_calificacion_libro
        FROM 
            books 
            INNER JOIN ratings ON books.book_id = ratings.book_id
    )
    SELECT DISTINCT  * FROM book_avg_rating;
"""
avg_book = execute_query(query2, engine, title="4.Calificación promedio para cada libro.")

query3 = """
    WITH editoriales_filtradas AS (
        SELECT 
            publishers.publisher,
            COUNT(books.book_id) AS cantidad_de_libros,
            SUM(books.num_pages) AS cantidad_de_paginas
        FROM 
            books
            INNER JOIN publishers ON books.publisher_id = publishers.publisher_id
        WHERE books.num_pages > 50  
        GROUP BY publishers.publisher
    )
    SELECT 
        ef.publisher AS editoriales,
        ef.cantidad_de_libros,
        ef.cantidad_de_paginas,
        DENSE_RANK() OVER (ORDER BY ef.cantidad_de_libros DESC, ef.cantidad_de_paginas DESC, ef.publisher ASC) AS ranking_editorial
    FROM 
        editoriales_filtradas ef
    ORDER BY 
        ranking_editorial, ef.cantidad_de_libros DESC, ef.cantidad_de_paginas DESC, ef.publisher ASC;
"""
editorial_book  = execute_query(query3, engine, title="4.Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).")




,total_de_libros
0,819


,username,reviews_cantidad
0,murrayerin,15
1,ehall,18
2,annawarner,18
3,tnolan,19
4,ncurry,16
...,...,...
155,urussell,21
156,vcarter,20
157,johnsonamanda,24
158,xdavis,18


,reviews_cantidad
0,2793


,nombre_libro,promedio_calificacion_libro
0,Stolen (Women of the Otherworld #2),5.000000
1,Ghost Story,2.666667
2,The Harlequin (Anita Blake Vampire Hunter #15),3.000000
3,The Secret History,3.777778
4,Last Chance Saloon,4.500000
...,...,...
994,Pop Goes the Weasel (Alex Cross #5),5.000000
995,You Suck (A Love Story #2),4.500000
996,The House of Mirth,4.000000
997,Far From the Madding Crowd,4.250000


,editoriales,cantidad_de_libros,cantidad_de_paginas,ranking_editorial
0,Penguin Books,42,15961,1
1,Vintage,31,10895,2
2,Grand Central Publishing,25,10631,3
3,Penguin Classics,24,12247,4
4,Ballantine Books,19,9495,5
...,...,...,...,...
329,Warne,1,72,330
330,Harpercollins Children's Books,1,64,331
331,HarperCollinsChildren’sBooks,1,62,332
332,Listening Library (Audio),1,61,333


### **Conclusiones:**

#### 1. Encuentra el número de libros publicados después del 1 de enero de 2000:
- Se encontró un total de 819  libros publicados despues del 1 enero de 2000.

#### 2. Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro:
  - Se encontró un total de 2793 reseñas y 999 libros en total con sus calificaciones promedios.

#### 3. Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).
- La editorial *Penguin Books* es la que ha puplicado el mayor número de libros con más de 50 páginas.

In [71]:
def execute_query(query, engine, title=None):
    """
    Ejecuta una consulta SQL y devuelve los resultados en un DataFrame, mostrando un título antes de los resultados.
    
    Parámetros:
        query (str): La consulta SQL a ejecutar.
        engine: Conexión SQLAlchemy a la base de datos.
        title (str, opcional): Título a mostrar antes de los resultados.
    
    Retorno:
        pd.DataFrame: DataFrame con los resultados de la consulta.
    """
    try:
        # Mostrar título en negrita y tamaño más grande
        display(HTML(f"<h2 style='color: #2C3E50; font-size: 24px; font-weight: bold;'>{title}</h2>"))
        
        df = pd.read_sql(query, con=engine)
        display(df)
        return df
    except Exception as e:
        print(f"Error al ejecutar la consulta: {e}")
        return pd.DataFrame()


query = """
    WITH libros_filtrados AS (
        SELECT 
            books.book_id,
            books.title,
            authors.author,
            AVG(ratings.rating) AS calificacion_promedio,
            COUNT(ratings.rating) AS cant_de_calificaciones
        FROM 
            authors
            INNER JOIN books ON books.author_id = authors.author_id
            INNER JOIN ratings ON ratings.book_id = books.book_id
        GROUP BY books.book_id, books.title, authors.author
        HAVING COUNT(ratings.rating) >= 50
    )
    SELECT 
        lf.title AS nombre_del_libro,
        lf.author AS autor,
        lf.calificacion_promedio,
        lf.cant_de_calificaciones,
        RANK() OVER (ORDER BY lf.calificacion_promedio DESC) AS ranking_autor
    FROM 
        libros_filtrados lf
    ORDER BY 
       ranking_autor, lf.calificacion_promedio DESC;


"""
num_calificacion = execute_query(query, engine, title="5.Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.")


query1 = """
    WITH usuarios_calificadores AS (
        SELECT 
            username
        FROM ratings
        GROUP BY username
        HAVING COUNT(book_id) > 50
    ),
    reviews_usuarios AS (
        SELECT 
            r.username,
            COUNT(r.review_id) AS total_reviews
        FROM reviews r
        INNER JOIN usuarios_calificadores uc ON r.username = uc.username
        GROUP BY r.username
    )
    SELECT 
        AVG(total_reviews) AS promedio_reviews_por_usuario
    FROM reviews_usuarios;

"""
promedio_reviews = execute_query(query1, engine, title="6. Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.")


,nombre_del_libro,autor,calificacion_promedio,cant_de_calificaciones,ranking_autor
0,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.414634,82,1
1,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. Tolkien,4.391892,74,2
2,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling/Mary GrandPré,4.287500,80,3
3,The Book Thief,Markus Zusak/Cao Xuân Việt Khương,4.264151,53,4
4,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.246575,73,5
5,Little Women,Louisa May Alcott,4.192308,52,6
6,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.186667,75,7
7,The Hobbit or There and Back Again,J.R.R. Tolkien,4.125000,88,8
8,The Lightning Thief (Percy Jackson and the Oly...,Rick Riordan,4.080645,62,9
9,Lord of the Flies,William Golding,3.901408,71,10


,promedio_reviews_por_usuario
0,24.333333


### **Conclusiones:**

#### 4.Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.
- El J.K. Rowling/Mary GrandPré	es el que tiene la más alta calificación promedio, con un total de 4.41. Con el libro titulado:
  Harry Potter and the Prisoner of Azkaban	

#### 5.Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.
- El promedio de reseñas de texto es de 24.33
